# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [8]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` from Ironhack's database into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [9]:
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [21]:
spent = orders.groupby("CustomerID")["amount_spent"].sum().sort_values(ascending=False)
display(spent)

CustomerID
14646    280206.02
18102    259657.30
17450    194550.79
16446    168472.50
14911    143825.06
12415    124914.53
14156    117379.63
17511     91062.38
16029     81024.84
12346     77183.60
16684     66653.56
14096     65164.79
13694     65039.62
15311     60767.90
13089     58825.83
17949     58510.48
15769     56252.72
15061     54534.14
14298     51527.30
14088     50491.81
15749     44534.30
12931     42055.96
17841     40991.57
15098     39916.50
13798     37153.85
16013     37130.60
16422     34684.40
12748     33719.73
15838     33643.08
17404     31906.82
           ...    
14589        39.75
12977        39.24
16659        39.12
18019        38.45
12855        38.10
16162        37.40
17816        36.56
15940        35.80
17831        35.40
16856        35.40
14576        35.40
15744        34.80
16765        34.00
17408        32.65
13120        30.60
15657        30.00
18268        25.50
17102        25.50
17986        20.80
16953        20.80
16093        17.00
1

In [47]:
df = pd.DataFrame()
df["amount_spent"]=spent
df.head()

,amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06


In [52]:
def label(m):
    if m > spent.quantile(0.95):
        return "VIP"
    elif (m<= spent.quantile(0.95)) and (m>= spent.quantile(0.75)):
        return "Pref"
    else: 
        return "Ordinary"
    
df["Label"]=spent.apply(label)
df.head()


,amount_spent,Label
CustomerID,,
14646,280206.02,VIP
18102,259657.30,VIP
17450,194550.79,VIP
16446,168472.50,VIP
14911,143825.06,VIP


In [53]:
df["Label"].value_counts()

Ordinary    3254
Pref         868
VIP          217
Name: Label, dtype: int64

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [56]:
merged = pd.merge(df, orders[["CustomerID", "Country"]], on="CustomerID").drop_duplicates()
merged.head()

,CustomerID,amount_spent,Label,Country
0,14646,280206.02,VIP,Netherlands
2080,18102,259657.30,VIP,United Kingdom
2511,17450,194550.79,VIP,United Kingdom
2848,16446,168472.50,VIP,United Kingdom
2851,14911,143825.06,VIP,EIRE


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [76]:
comb = merged[(merged["Label"]=="VIP") | (merged["Label"]=="Pref")]
comb

,CustomerID,amount_spent,Label,Country
0,14646,280206.02,VIP,Netherlands
2080,18102,259657.30,VIP,United Kingdom
2511,17450,194550.79,VIP,United Kingdom
2848,16446,168472.50,VIP,United Kingdom
2851,14911,143825.06,VIP,EIRE
8528,12415,124914.53,VIP,Australia
9244,14156,117379.63,VIP,EIRE
10644,17511,91062.38,VIP,United Kingdom
11607,16029,81024.84,VIP,United Kingdom
11849,12346,77183.60,VIP,United Kingdom


In [78]:
comb.Country.value_counts()

United Kingdom     932
Germany             39
France              29
Belgium             12
Switzerland          9
Spain                9
Norway               7
Portugal             7
Finland              5
Italy                5
Cyprus               4
Japan                4
Channel Islands      4
Australia            4
EIRE                 3
Denmark              3
Austria              3
Israel               2
Sweden               2
Singapore            1
Canada               1
Malta                1
Iceland              1
Netherlands          1
Poland               1
Greece               1
Lebanon              1
Name: Country, dtype: int64